In [ ]:
# CFB Dynasty Roster Analysis and Visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import glob
import os
import warnings

# Dynasty analysis modules
from dynasty.analysis.player_valuation import calculate_player_value
from dynasty.analysis.roster_analysis import player_status, calculate_position_grade, calculate_blended_measure
from dynasty.analysis.recruiting_plan import determine_priority
from dynasty.config.constants import DEV_TRAIT_MULTIPLIERS, DEFAULT_POSITION_REQUIREMENTS, STARTERS_COUNT

# Utility modules
from utils.file_utils import load_roster, export_files

# Geography modules
from geography.heatmaps import create_geographic_heatmap, create_city_bar_chart
from geography.territory_analysis import create_recruiting_territory_map
from geography.geocoding import get_city_coordinates

# Import coordinate caching functions
try:
    from geography.geocoding import get_cache_statistics, clear_coordinate_cache
    coordinate_caching_available = True
except ImportError:
    coordinate_caching_available = False
    print("⚠️  Note: Advanced coordinate caching functions not available")

warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("✅ Libraries imported successfully!")
print("📊 Ready for CFB Dynasty roster analysis and visualization")
if coordinate_caching_available:
    print("🗺️ Geographic coordinate lookup with JSON file caching enabled")
else:
    print("🗺️ Basic geographic coordinate lookup enabled")

# 🏈 CFB Dynasty Roster Analysis Dashboard

This notebook provides comprehensive analysis and visualization of your CFB Dynasty roster data, including:

- **Player Valuations** - Calculate player value based on ratings, development traits, and remaining years
- **Roster Composition** - Analyze position strength and depth across your roster
- **Recruiting Strategy** - Identify positions of need and priority for recruiting
- **Interactive Visualizations** - Charts and graphs to better understand your roster

---

In [ ]:
# Configuration and Constants



print("📋 Configuration loaded successfully!")
print(f"📊 Tracking {len(DEFAULT_POSITION_REQUIREMENTS)} positions")
print(f"🎯 {len(DEV_TRAIT_MULTIPLIERS)} development trait levels defined")

In [ ]:






print("✅ Analysis functions defined successfully!")
print("🔧 Ready to process roster data")

## 📂 Data Loading

Load your roster CSV file from the Downloads folder. The script will automatically detect roster files.

In [ ]:
# Load Roster Data
roster_df = load_roster()


In [ ]:
# Process Roster Data
if roster_df is not None:
    # Ensure required columns exist
    required_columns = [
        'RS', 'FIRST NAME', 'LAST NAME', 'YEAR', 'POSITION', 'OVERALL', 'BASE OVERALL',
        'ARCHETYPE', 'DEV TRAIT', 'VALUE', 'STATUS', 'Best at Position', 'Current Count', 'Min Required', 'Grade'
    ]

    missing_columns = [col for col in required_columns if col not in roster_df.columns]
    if missing_columns:
        print(f"⚠️  Missing required columns: {missing_columns}")
        print("📝 Adding missing columns with default values...")

        # Add missing columns with defaults
        for col in missing_columns:
            if col in ['CUT', 'REDSHIRT']:
                roster_df[col] = False
            elif col == 'DRAFTED':
                roster_df[col] = None
            else:
                roster_df[col] = ''

    # Calculate player values
    roster_df['VALUE'] = roster_df.apply(lambda row: calculate_player_value(row), axis=1)

    # Default handling for empty RS values
    roster_df['RS'] = roster_df['RS'].fillna('')

    # Fill missing archetypes
    roster_df['ARCHETYPE'] = roster_df['ARCHETYPE'].fillna('')

    # Determine best player at each position
    roster_df['Best at Position'] = roster_df.groupby('POSITION')['OVERALL'].transform(lambda x: x == x.max())

    # Apply player status
    roster_df['STATUS'] = roster_df.apply(player_status, axis=1)

    # Sort roster
    position_order = ['QB', 'HB', 'WR', 'TE', 'LT', 'LG', 'C', 'RG', 'RT',
                     'LEDG', 'REDG', 'DT', 'WILL', 'MIKE', 'SAM', 'CB', 'FS', 'SS', 'K', 'P', 'ATH']
    roster_df['POSITION'] = pd.Categorical(roster_df['POSITION'], categories=position_order, ordered=True)
    roster_df = roster_df.sort_values(by=['POSITION', 'OVERALL'], ascending=[True, False])

    print("✅ Roster data processed successfully!")
    print(f"💰 Player values calculated")
    print(f"📊 Player status determined")

    # Show sample of processed data
    print(f"\n📋 Sample of processed roster:")
    display_cols = [
        'RS', 'FIRST NAME', 'LAST NAME', 'YEAR', 'POSITION', 'OVERALL',
        'BASE OVERALL', 'ARCHETYPE', 'DEV TRAIT', 'VALUE', 'STATUS'
    ]
    sample_df = roster_df[display_cols].head(10)
    print(sample_df.to_string(index=False))

else:
    print("❌ Failed to load roster data. Please check the file path and format.")
    exit(1)

In [ ]:
# Generate Recruiting Plan
if roster_df is not None:
    # Calculate next season counts (excluding graduating players)
    next_season_counts = roster_df[roster_df['STATUS'] != 'GRADUATING'].groupby('POSITION').size()

    # Calculate blended values for each position
    blended_values = {pos: calculate_blended_measure(roster_df, pos) for pos in DEFAULT_POSITION_REQUIREMENTS.keys()}

    # Create recruiting plan DataFrame
    recruiting_plan = pd.DataFrame({
        'Position': DEFAULT_POSITION_REQUIREMENTS.keys(),
        'Current Count': [next_season_counts.get(pos, 0) for pos in DEFAULT_POSITION_REQUIREMENTS.keys()],
        'Min Required': [DEFAULT_POSITION_REQUIREMENTS[pos]['min'] for pos in DEFAULT_POSITION_REQUIREMENTS.keys()],
        'Ideal Count': [DEFAULT_POSITION_REQUIREMENTS[pos]['ideal'] for pos in DEFAULT_POSITION_REQUIREMENTS.keys()],
        'Blended Value': [blended_values[pos] for pos in DEFAULT_POSITION_REQUIREMENTS.keys()],
        'Grade': [calculate_position_grade(blended_values[pos]) for pos in DEFAULT_POSITION_REQUIREMENTS.keys()]
    }).fillna(0)

    # Determine priority
    recruiting_plan['Priority'] = recruiting_plan.apply(determine_priority, axis=1)

    # Calculate need (difference between current and ideal)
    recruiting_plan['Need'] = recruiting_plan['Ideal Count'] - recruiting_plan['Current Count']
    recruiting_plan['Need'] = recruiting_plan['Need'].clip(lower=0)  # Don't show negative needs

    print("✅ Recruiting plan generated!")
    print(f"\n📋 Recruiting Priorities Summary:")

    # Show high priority positions
    high_priority = recruiting_plan[recruiting_plan['Priority'] == 'HIGH'].sort_values('Need', ascending=False)
    if len(high_priority) > 0:
        print(f"\n🚨 HIGH PRIORITY positions ({len(high_priority)}):")
        for _, row in high_priority.iterrows():
            print(f"  • {row['Position']}: {int(row['Current Count'])}/{int(row['Min Required'])} (Grade: {row['Grade']})")

    medium_priority = recruiting_plan[recruiting_plan['Priority'] == 'MEDIUM']
    if len(medium_priority) > 0:
        print(f"\n⚠️  MEDIUM PRIORITY positions ({len(medium_priority)}):")
        for _, row in medium_priority.iterrows():
            print(f"  • {row['Position']}: Grade {row['Grade']} (Value: {row['Blended Value']:.1f})")

    print(f"\n📊 Full recruiting plan available in 'recruiting_plan' DataFrame")

## 📊 Data Visualizations

Interactive charts and graphs to analyze your roster composition, player values, and recruiting needs.

In [ ]:
# 1. Roster Composition by Position
if roster_df is not None:
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Current Roster by Position', 'Position Strength Grades',
                       'Player Status Distribution', 'Development Traits'),
        specs=[[{"type": "bar"}, {"type": "bar"}],
               [{"type": "pie"}, {"type": "pie"}]]
    )

    # Subplot 1: Current roster by position
    position_counts = roster_df['POSITION'].value_counts().reindex(position_order).fillna(0)
    fig.add_trace(
        go.Bar(x=position_counts.index, y=position_counts.values,
               name="Current Count", marker_color='lightblue'),
        row=1, col=1
    )

    # Subplot 2: Position grades
    grade_colors = {'A+': '#2E8B57', 'A': '#32CD32', 'A-': '#90EE90',
                   'B+': '#FFD700', 'B': '#FFA500', 'B-': '#FF8C00',
                   'C+': '#FF6347', 'C': '#FF4500', 'C-': '#DC143C', 'F': '#8B0000'}

    recruiting_plan_sorted = recruiting_plan.sort_values('Blended Value', ascending=True)
    colors = [grade_colors.get(grade, '#808080') for grade in recruiting_plan_sorted['Grade']]

    fig.add_trace(
        go.Bar(x=recruiting_plan_sorted['Blended Value'],
               y=recruiting_plan_sorted['Position'],
               orientation='h',
               name="Position Strength",
               marker_color=colors,
               text=recruiting_plan_sorted['Grade'],
               textposition='auto'),
        row=1, col=2
    )

    # Subplot 3: Player status distribution
    status_counts = roster_df['STATUS'].value_counts()
    status_colors = {'SAFE': '#2E8B57', 'AT RISK': '#FFD700', 'CUT': '#DC143C', 'GRADUATING': '#808080'}
    fig.add_trace(
        go.Pie(labels=status_counts.index, values=status_counts.values,
               name="Status", marker_colors=[status_colors.get(status, '#808080') for status in status_counts.index]),
        row=2, col=1
    )

    # Subplot 4: Development traits
    dev_counts = roster_df['DEV TRAIT'].value_counts()
    dev_colors = {'ELITE': '#2E8B57', 'STAR': '#32CD32', 'IMPACT': '#FFD700', 'NORMAL': '#808080'}
    fig.add_trace(
        go.Pie(labels=dev_counts.index, values=dev_counts.values,
               name="Dev Traits", marker_colors=[dev_colors.get(trait, '#808080') for trait in dev_counts.index]),
        row=2, col=2
    )

    fig.update_layout(height=800, title_text="🏈 Roster Overview Dashboard", showlegend=False)
    fig.update_xaxes(title_text="Position", row=1, col=1)
    fig.update_yaxes(title_text="Player Count", row=1, col=1)
    fig.update_xaxes(title_text="Blended Value", row=1, col=2)

    fig.show()

    print("📊 Roster composition visualization created!")

In [ ]:
# 2. Player Value Distribution
if roster_df is not None:
    fig = px.box(roster_df, x='POSITION', y='VALUE',
                 color='STATUS',
                 title='📈 Player Value Distribution by Position',
                 color_discrete_map={'SAFE': '#2E8B57', 'AT RISK': '#FFD700',
                                   'CUT': '#DC143C', 'GRADUATING': '#808080'})

    fig.update_layout(
        xaxis_title="Position",
        yaxis_title="Player Value",
        height=600,
        xaxis={'categoryorder':'array', 'categoryarray':position_order}
    )

    fig.show()

    # Additional value statistics
    print(f"💰 Player Value Statistics:")
    print(f"  • Average Value: {roster_df['VALUE'].mean():.1f}")
    print(f"  • Highest Value: {roster_df['VALUE'].max():.1f} ({roster_df.loc[roster_df['VALUE'].idxmax(), 'FIRST NAME']} {roster_df.loc[roster_df['VALUE'].idxmax(), 'LAST NAME']})")
    print(f"  • Lowest Value: {roster_df['VALUE'].min():.1f}")
    print(f"  • Players > 150 Value: {len(roster_df[roster_df['VALUE'] > 150])}")
    print(f"  • Players at Risk (100-125): {len(roster_df[roster_df['STATUS'] == 'AT RISK'])}")
    print(f"  • Players to Cut (<100): {len(roster_df[roster_df['STATUS'] == 'CUT'])}")

In [ ]:
# 3. Recruiting Priority Heatmap
if roster_df is not None and recruiting_plan is not None:
    # Create a matrix for the heatmap
    priority_map = {'HIGH': 3, 'MEDIUM': 2, 'LOW': 1}
    recruiting_plan['Priority_Numeric'] = recruiting_plan['Priority'].map(priority_map)

    # Create heatmap data
    heatmap_data = recruiting_plan.pivot_table(
        index='Position',
        values=['Priority_Numeric', 'Need', 'Blended Value'],
        aggfunc='first'
    ).fillna(0)

    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=('Recruiting Priority', 'Position Need', 'Current Strength'),
        horizontal_spacing=0.1
    )

    # Priority heatmap
    fig.add_trace(
        go.Heatmap(
            z=[heatmap_data['Priority_Numeric'].values],
            x=heatmap_data.index,
            y=['Priority'],
            colorscale=[[0, 'green'], [0.5, 'yellow'], [1, 'red']],
            showscale=False,
            text=[[recruiting_plan.set_index('Position').loc[pos, 'Priority'] for pos in heatmap_data.index]],
            texttemplate="%{text}",
            textfont={"size": 10}
        ),
        row=1, col=1
    )

    # Need heatmap
    fig.add_trace(
        go.Heatmap(
            z=[heatmap_data['Need'].values],
            x=heatmap_data.index,
            y=['Need'],
            colorscale='Reds',
            showscale=False,
            text=[[f"{int(val)}" for val in heatmap_data['Need'].values]],
            texttemplate="%{text}",
            textfont={"size": 10}
        ),
        row=1, col=2
    )

    # Strength heatmap
    fig.add_trace(
        go.Heatmap(
            z=[heatmap_data['Blended Value'].values],
            x=heatmap_data.index,
            y=['Strength'],
            colorscale='RdYlGn',
            showscale=True,
            text=[[f"{val:.0f}" for val in heatmap_data['Blended Value'].values]],
            texttemplate="%{text}",
            textfont={"size": 10}
        ),
        row=1, col=3
    )

    fig.update_layout(
        title='🎯 Recruiting Strategy Heatmap',
        height=400,
        showlegend=False
    )

    # Rotate x-axis labels
    for i in range(1, 4):
        fig.update_xaxes(tickangle=45, row=1, col=i)

    fig.show()

    print("🎯 Recruiting priority heatmap created!")
    print("\n📋 Quick Reference:")
    print("  🔴 RED = High Priority/High Need")
    print("  🟡 YELLOW = Medium Priority")
    print("  🟢 GREEN = Low Priority/Strong Position")

In [ ]:
# 4. Year Class Distribution and Future Outlook
if roster_df is not None:
    # Create year class analysis
    year_order = ['FR', 'SO', 'JR', 'SR', 'FR (RS)', 'SO (RS)', 'JR (RS)', 'SR (RS)']

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Current Year Distribution', 'Position by Year Class',
                       'Development Potential by Year', 'Future Roster Projection'),
        specs=[[{"type": "bar"}, {"type": "scatter"}],
               [{"type": "bar"}, {"type": "bar"}]]
    )

    # Year distribution
    year_counts = roster_df['YEAR'].value_counts().reindex(year_order).fillna(0)
    fig.add_trace(
        go.Bar(x=year_counts.index, y=year_counts.values,
               name="Players by Year", marker_color='lightcoral'),
        row=1, col=1
    )

    # Position by year (scatter plot)
    for pos in position_order[:10]:  # Show top 10 positions to avoid clutter
        pos_data = roster_df[roster_df['POSITION'] == pos]
        if len(pos_data) > 0:
            fig.add_trace(
                go.Scatter(x=pos_data['YEAR'], y=[pos] * len(pos_data),
                          mode='markers', name=pos,
                          marker=dict(size=8, opacity=0.7)),
                row=1, col=2
            )

    # Development potential by year
    year_dev_potential = roster_df.groupby('YEAR')['VALUE'].mean().reindex(year_order).fillna(0)
    fig.add_trace(
        go.Bar(x=year_dev_potential.index, y=year_dev_potential.values,
               name="Avg Value by Year", marker_color='lightgreen'),
        row=2, col=1
    )

    # Future projection (non-graduating players)
    future_roster = roster_df[roster_df['STATUS'] != 'GRADUATING']
    future_counts = future_roster['POSITION'].value_counts().reindex(position_order).fillna(0)
    fig.add_trace(
        go.Bar(x=future_counts.index, y=future_counts.values,
               name="Next Season Projection", marker_color='gold'),
        row=2, col=2
    )

    fig.update_layout(height=800, title_text="📅 Year Class Analysis & Future Outlook", showlegend=False)
    fig.update_xaxes(title_text="Year", row=1, col=1)
    fig.update_yaxes(title_text="Player Count", row=1, col=1)
    fig.update_xaxes(title_text="Year", row=1, col=2)
    fig.update_yaxes(title_text="Position", row=1, col=2)
    fig.update_xaxes(title_text="Year", row=2, col=1)
    fig.update_yaxes(title_text="Average Value", row=2, col=1)
    fig.update_xaxes(title_text="Position", row=2, col=2, tickangle=45)
    fig.update_yaxes(title_text="Player Count", row=2, col=2)

    fig.show()

    # Print year class insights
    print("📅 Year Class Insights:")
    graduating = len(roster_df[roster_df['STATUS'] == 'GRADUATING'])
    total_players = len(roster_df)
    print(f"  • {graduating} players graduating ({graduating/total_players*100:.1f}% of roster)")
    print(f"  • {len(future_roster)} players returning next season")

    # Show positions most affected by graduation
    grad_by_pos = roster_df[roster_df['STATUS'] == 'GRADUATING']['POSITION'].value_counts()
    if len(grad_by_pos) > 0:
        print(f"  • Positions losing most players: {', '.join(grad_by_pos.head(3).index.tolist())}")

In [ ]:
# 6. Geographic Heatmap - Player Distribution by Location
# Enhanced geographic analysis with automatic geocoding and recruiting territory mapping

# Import required plotly modules if not already available
try:
    # Test if plotly modules are already imported
    go
    px
except NameError:
    # Import plotly modules
    try:
        import plotly.graph_objects as go
        import plotly.express as px
        print("📦 Plotly modules imported successfully!")
    except ImportError:
        print("❌ Plotly not installed. Installing now...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'plotly'])
        import plotly.graph_objects as go
        import plotly.express as px
        print("✅ Plotly installed and imported successfully!")

# Geographic analysis using the new geography module
if roster_df is not None:
    # Check if geographic data exists
    has_geographic_data = 'CITY' in roster_df.columns and 'STATE' in roster_df.columns

    if has_geographic_data:
        print("✅ Geographic data found! Creating heatmaps...")
        create_geographic_heatmap(roster_df)
        create_recruiting_territory_map(roster_df)
    else:
        print("📊 Geographic Heatmap Demo")
        print("🗺️ To enable geographic visualization, add these columns to your roster CSV:")
        print("   • CITY - Player's hometown city")
        print("   • STATE - Player's home state (use 2-letter abbreviations: TX, CA, FL, etc.)")
        print()
        print("📝 Example data structure:")
        sample_data = pd.DataFrame({
            'FIRST NAME': ['John', 'Mike', 'David', 'Chris', 'Alex'],
            'LAST NAME': ['Smith', 'Johnson', 'Williams', 'Brown', 'Davis'],
            'POSITION': ['QB', 'RB', 'WR', 'LB', 'CB'],
            'CITY': ['Atlanta', 'Dallas', 'Miami', 'Houston', 'Los Angeles'],
            'STATE': ['GA', 'TX', 'FL', 'TX', 'CA'],
            'VALUE': [145.2, 132.1, 128.5, 142.8, 139.4],
            'DEV TRAIT': ['ELITE', 'STAR', 'IMPACT', 'STAR', 'ELITE']
        })
        print(sample_data.to_string(index=False))
        print()
        print("🔧 Once you have this data, the geographic heatmaps will automatically generate!")
        print("📍 Features include:")
        print("   • US State heatmap showing player distribution")
        print("   • Automatic city geocoding and mapping")
        print("   • Top cities by player count")
        print("   • Recruiting territory success analysis")
        print("   • Average player value by state")

In [ ]:
# 5. Top Performers and Cut Candidates Analysis
if roster_df is not None:
    # Create top performers vs cut candidates comparison
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Top 15 Players by Value', 'Development Trait vs Rating',
                       'Cut Candidates', 'At-Risk Players'),
        specs=[[{"type": "bar"}, {"type": "scatter"}],
               [{"type": "table"}, {"type": "table"}]]
    )

    # Top performers
    top_players = roster_df.nlargest(15, 'VALUE')
    player_names = top_players['FIRST NAME'] + ' ' + top_players['LAST NAME']

    fig.add_trace(
        go.Bar(x=player_names, y=top_players['VALUE'],
               name="Top Players",
               marker_color=top_players['VALUE'],
               marker_colorscale='Viridis',
               text=top_players['POSITION'],
               textposition='auto'),
        row=1, col=1
    )

    # Development trait vs rating scatter
    dev_trait_colors = {'ELITE': '#2E8B57', 'STAR': '#32CD32', 'IMPACT': '#FFD700', 'NORMAL': '#808080'}
    for trait in roster_df['DEV TRAIT'].unique():
        trait_data = roster_df[roster_df['DEV TRAIT'] == trait]
        fig.add_trace(
            go.Scatter(x=trait_data['BASE OVERALL'], y=trait_data['VALUE'],
                      mode='markers', name=trait,
                      marker=dict(color=dev_trait_colors.get(trait, '#808080'), size=8)),
            row=1, col=2
        )

    # Cut candidates table
    cut_candidates = roster_df[roster_df['STATUS'] == 'CUT'][
        ['FIRST NAME', 'LAST NAME', 'POSITION', 'YEAR', 'BASE OVERALL', 'VALUE']
    ].round(1)

    if len(cut_candidates) > 0:
        fig.add_trace(
            go.Table(
                header=dict(values=list(cut_candidates.columns), fill_color='lightcoral'),
                cells=dict(values=[cut_candidates[col] for col in cut_candidates.columns],
                          fill_color='white')
            ),
            row=2, col=1
        )

    # At-risk players table
    at_risk = roster_df[roster_df['STATUS'] == 'AT RISK'][
        ['FIRST NAME', 'LAST NAME', 'POSITION', 'YEAR', 'BASE OVERALL', 'VALUE']
    ].round(1)

    if len(at_risk) > 0:
        fig.add_trace(
            go.Table(
                header=dict(values=list(at_risk.columns), fill_color='gold'),
                cells=dict(values=[at_risk[col] for col in at_risk.columns],
                          fill_color='white')
            ),
            row=2, col=2
        )

    fig.update_layout(height=800, title_text="⭐ Player Performance Analysis", showlegend=True)
    fig.update_xaxes(title_text="Player", row=1, col=1, tickangle=45)
    fig.update_yaxes(title_text="Player Value", row=1, col=1)
    fig.update_xaxes(title_text="Base Overall", row=1, col=2)
    fig.update_yaxes(title_text="Player Value", row=1, col=2)

    fig.show()

    # Print key insights
    print("⭐ Performance Insights:")
    print(f"  • Top Player: {roster_df.loc[roster_df['VALUE'].idxmax(), 'FIRST NAME']} {roster_df.loc[roster_df['VALUE'].idxmax(), 'LAST NAME']} ({roster_df['VALUE'].max():.1f} value)")
    print(f"  • Elite Dev Traits: {len(roster_df[roster_df['DEV TRAIT'] == 'ELITE'])} players")
    print(f"  • Star Dev Traits: {len(roster_df[roster_df['DEV TRAIT'] == 'STAR'])} players")
    print(f"  • Players to Cut: {len(cut_candidates)}")
    print(f"  • Players At Risk: {len(at_risk)}")

## 💾 Export Results

Save your analysis results to CSV files for future reference or sharing.

In [ ]:
# Export Analysis Results
if roster_df is not None and recruiting_plan is not None:
    # Use the export_files function from file_utils
    success = export_files(
        roster_df=roster_df,
        recruiting_plan=recruiting_plan,
        position_requirements=DEFAULT_POSITION_REQUIREMENTS
    )

    if not success:
        print("❌ Export failed. Please check the error messages above.")
else:
    print("❌ Cannot export - no data available. Please run the analysis cells above first.")

## 🔍 Interactive Data Exploration

Use these cells to explore your data interactively. Modify the code below to filter and analyze specific aspects of your roster.

In [ ]:
# Interactive Exploration Examples

# Example 1: Filter by specific position (change 'QB' to any position)
if roster_df is not None:
    position_to_analyze = 'QB'  # Change this to any position
    position_players = roster_df[roster_df['POSITION'] == position_to_analyze].copy()

    if len(position_players) > 0:
        print(f"📊 {position_to_analyze} Analysis:")
        print(f"  • Total Players: {len(position_players)}")
        print(f"  • Average Rating: {position_players['BASE OVERALL'].mean():.1f}")
        print(f"  • Average Value: {position_players['VALUE'].mean():.1f}")
        print(f"  • Top Player: {position_players.loc[position_players['VALUE'].idxmax(), 'FIRST NAME']} {position_players.loc[position_players['VALUE'].idxmax(), 'LAST NAME']} ({position_players['VALUE'].max():.1f})")

        # Show detailed breakdown
        print(f"\n📋 Detailed {position_to_analyze} Roster:")
        display_cols = ['RS', 'FIRST NAME', 'LAST NAME', 'YEAR', 'OVERALL', 'BASE OVERALL', 'ARCHETYPE', 'DEV TRAIT', 'VALUE', 'STATUS']
        print(position_players[display_cols].sort_values('VALUE', ascending=False).to_string(index=False))
    else:
        print(f"❌ No players found at {position_to_analyze} position")
else:
    print("❌ No roster data available. Please run the data loading cells first.")

In [ ]:
# Example 2: Custom Filtering - High Value Underclassmen
if roster_df is not None:
    # Find high-value underclassmen (FR/SO with value > 120)
    underclassmen = roster_df[
        (roster_df['YEAR'].isin(['FR', 'SO', 'FR (RS)', 'SO (RS)'])) &
        (roster_df['VALUE'] > 120)
    ].copy()

    if len(underclassmen) > 0:
        print("⭐ High-Value Underclassmen (Future Stars):")
        print(f"Found {len(underclassmen)} underclassmen with value > 120")

        # Create visualization
        fig = px.scatter(underclassmen, x='BASE OVERALL', y='VALUE',
                        color='DEV TRAIT', size='OVERALL',
                        hover_data=['FIRST NAME', 'LAST NAME', 'POSITION', 'YEAR'],
                        title='🌟 Future Stars: High-Value Underclassmen')
        fig.show()

        # Show top prospects
        top_prospects = underclassmen.nlargest(10, 'VALUE')
        display_cols = ['FIRST NAME', 'LAST NAME', 'POSITION', 'YEAR', 'BASE OVERALL', 'DEV TRAIT', 'VALUE']
        print(f"\n🏆 Top 10 Future Prospects:")
        print(top_prospects[display_cols].to_string(index=False))
    else:
        print("🔍 No high-value underclassmen found (try lowering the value threshold)")

# Example 3: Position Group Analysis
if roster_df is not None:
    print(f"\n📊 Position Group Summary:")

    # Group positions by unit
    position_groups = {
        'Offense': ['QB', 'HB', 'FB', 'WR', 'TE', 'LT', 'LG', 'C', 'RG', 'RT'],
        'Defense': ['LEDG', 'REDG', 'DT', 'WILL', 'MIKE', 'SAM', 'CB', 'FS', 'SS'],
        'Special Teams': ['K', 'P']
    }

    for group_name, positions in position_groups.items():
        group_players = roster_df[roster_df['POSITION'].isin(positions)]
        if len(group_players) > 0:
            avg_value = group_players['VALUE'].mean()
            print(f"  {group_name}: {len(group_players)} players, Avg Value: {avg_value:.1f}")
        else:
            print(f"  {group_name}: No players found")

## 📋 Summary & Next Steps

This notebook has provided a comprehensive analysis of your CFB Dynasty roster including:

- ✅ **Player Valuations** - Each player's calculated value based on ratings, development potential, and remaining years
- ✅ **Roster Composition** - Visual breakdown of your team's strengths and weaknesses by position
- ✅ **Recruiting Strategy** - Data-driven priorities for recruiting based on position needs and current grades
- ✅ **Future Outlook** - Analysis of year classes and projected roster changes
- ✅ **Performance Insights** - Identification of top performers and players at risk

### 🎯 Key Actions to Take:

1. **Review High Priority Positions** - Focus recruiting efforts on positions marked as HIGH priority
2. **Evaluate Cut Candidates** - Consider cutting players with status "CUT" to make roster space
3. **Monitor At-Risk Players** - Keep an eye on players with status "AT RISK" for potential development
4. **Plan for Graduation** - Prepare for the loss of graduating seniors by recruiting replacements

### 📁 Exported Files:
- `player_values_analysis.csv` - Complete roster with calculated values and status
- `recruiting_plan_analysis.csv` - Position-by-position recruiting priorities  
- `position_analysis_detailed.csv` - Detailed breakdown of each position group

---

*This analysis was generated using the CFB Dynasty Roster Analysis notebook. Modify the parameters and filters above to customize the analysis for your specific needs.*